In [3]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input, EfficientNetB0
import cv2
from ultralytics import YOLO

# --- 설정 ---
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5,
}

def set_seed(seed=CFG['SEED']):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed()

# 경로
ORIGINAL_TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/filtered_train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)

# GPU 확인
print("GPU 사용 가능:", bool(tf.config.list_physical_devices('GPU')))

# --- YOLOv8 차량 외관 필터링 ---
yolo_model = YOLO('yolov8n.pt')
VEHICLE_CLASSES = [2, 5, 7]  # car, bus, truck

def is_full_vehicle(detections, img):
    h, w = img.shape[:2]
    img_area = h * w
    for det in detections:
        cls = int(det.cls)
        if cls not in VEHICLE_CLASSES:
            continue
        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        box_w, box_h = x2 - x1, y2 - y1
        area = box_w * box_h
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        aspect_ratio = box_w / box_h
        if not (0.35 * w < center_x < 0.65 * w and 0.35 * h < center_y < 0.65 * h):
            continue
        if area / img_area < 0.35:
            continue
        if aspect_ratio < 0.8 or aspect_ratio > 2.5:
            continue
        return True
    return False

print("차량 외관 필터링 시작...")
for cls in tqdm(os.listdir(ORIGINAL_TRAIN_DIR), desc="클래스별 필터링"):
    input_dir = os.path.join(ORIGINAL_TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_TRAIN_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)
    
    for img_path in glob(os.path.join(input_dir, '*.jpg')):
        img = cv2.imread(img_path)
        results = yolo_model(img, verbose=False)[0]
        if is_full_vehicle(results.boxes, img):
            cv2.imwrite(os.path.join(output_dir, os.path.basename(img_path)), img)

# --- 필터링된 데이터셋 준비 ---
image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# 데이터 증강
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

def load_and_preprocess(img_path, augment=False):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    if augment:
        img = data_augmentation(img)
    img = preprocess_input(img)
    return img

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path, augment=is_train)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img
    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )
    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# --- 모델 생성 ---
def build_model(num_classes):
    base = EfficientNetB0(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    x = layers.Dense(num_classes, activation='softmax')(base.output)
    return models.Model(inputs=base.input, outputs=x)

# --- Stratified K-Fold 학습 ---
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1} 시작 ###")
    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    model.fit(train_ds, validation_data=val_ds, epochs=CFG['EPOCHS'], verbose=1)

# --- 테스트 데이터 추론 ---
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

preds = model.predict(test_ds)

submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]
submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")

# --- 예측 결과 시각화 ---
import matplotlib.pyplot as plt

for i in range(5):
    img = load_img(test_paths[i], target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    plt.imshow(img)
    pred_class = label_list[np.argmax(preds[i])]
    plt.title(f"예측: {pred_class}")
    plt.axis('off')
    plt.show()



TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates